In [191]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
import string
import copy

[nltk_data] Downloading package stopwords to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [192]:
data_dir = '../Humor,Hist,Media,Food'
file_names = os.listdir(data_dir)
file_paths = [(data_dir + '/' + fname) for fname in file_names]

In [193]:
stop_words = set(stopwords.words('english'))

def read_files(fpaths):
    file_tokens = []
    for fpath in fpaths:
        f = open(fpath, 'r', encoding='utf-8', errors='replace')
        ftxt_unprocessed = f.read()
        # print(ftxt_unprocessed)
        ftoks = preprocess(ftxt_unprocessed)
        file_tokens.append(ftoks)
    return file_tokens

def isValidTok(tok):
    if((tok not in string.punctuation) and (tok.isnumeric() == False) and (sum([0 if ch in string.punctuation else 1 for ch in tok]) >= 1)):
        return True
    return False 

def preprocess(file_text):

    all_tokens = word_tokenize(file_text.lower())
    all_unique_tokens = set(all_tokens)
    tokens = list(all_unique_tokens - stop_words)
    # ps = PorterStemmer()
    valid_toks = []
    for tok in tokens:
        if(isValidTok(tok) == True):
            valid_toks.append(tok)
    return valid_toks
    # print(final_tokens)
    

In [194]:
file_toks = read_files(file_paths)

In [195]:
def create_inverted_index(file_toks):
    inv_index = {}
    for i in range(len(file_toks)):
        for tok in file_toks[i]:
            if(tok not in inv_index.keys()):
                inv_index[tok] = [i]
            else:
                inv_index[tok].append(i)
    inv_index = dict(sorted(inv_index.items()))
    terms_list = inv_index.keys()
    for word in terms_list:
        inv_index[word].sort()
    return inv_index

In [ ]:
def getDocsFromID(file_names, doc_IDs):
    doc_names = []
    for doc_ID in doc_IDs:
        doc_names.append(file_names[doc_ID])
    return doc_names

In [212]:
def query_AND(inv_index, term1, term2):

    terms_list = inv_index.keys()

    if((term1 not in terms_list) or (term2 not in terms_list)):
        return []

    posting1 = inv_index[term1]
    posting2 = inv_index[term2]
    ptr1 = 0
    ptr2 = 0
    answer = []

    num_comparisons = 0

    while(ptr1 < len(posting1) and ptr2 < len(posting2)):
        num_comparisons += 1
        # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")

        if(posting1[ptr1] == posting2[ptr2]):
            answer.append(posting1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif(posting1[ptr1] < posting2[ptr2]):
            ptr1 += 1
        else:
            ptr2 += 1
    return num_comparisons, answer
    

In [197]:
inverted_index = create_inverted_index(file_toks)

In [210]:
print(inverted_index['water'])

[0, 1, 9, 18, 21, 29, 34, 35, 38, 43, 44, 46, 51, 55, 56, 58, 60, 63, 64, 67, 69, 70, 74, 75, 85, 91, 97, 106, 108, 109, 121, 123, 124, 127, 128, 129, 131, 133, 134, 138, 139, 140, 145, 146, 147, 149, 150, 152, 156, 159, 167, 168, 169, 174, 175, 178, 179, 181, 184, 186, 187, 214, 219, 225, 226, 228, 229, 232, 233, 234, 235, 239, 240, 246, 252, 256, 257, 268, 282, 284, 288, 297, 300, 315, 316, 321, 324, 328, 329, 330, 331, 340, 343, 344, 349, 351, 352, 354, 360, 362, 365, 367, 369, 372, 376, 378, 379, 382, 384, 385, 390, 391, 392, 393, 397, 399, 400, 401, 404, 405, 408, 418, 429, 430, 431, 436, 439, 441, 443, 450, 453, 455, 457, 470, 479, 483, 488, 489, 494, 501, 504, 510, 511, 513, 517, 518, 519, 520, 531, 532, 533, 534, 537, 538, 541, 543, 546, 547, 548, 549, 552, 557, 565, 574, 579, 584, 594, 601, 608, 610, 611, 622, 625, 630, 632, 635, 641, 642, 648, 649, 658, 662, 663, 668, 670, 672, 674, 687, 701, 706, 707, 713, 715, 719, 724, 730, 734, 738, 744, 747, 748, 749, 752, 760, 762, 764,

In [207]:
print(inverted_index['effect'])

[0, 4, 6, 9, 35, 42, 44, 55, 57, 58, 63, 67, 73, 88, 89, 119, 120, 121, 127, 135, 137, 140, 173, 175, 184, 190, 191, 192, 193, 197, 200, 228, 231, 243, 247, 250, 253, 259, 267, 278, 281, 286, 289, 295, 315, 324, 339, 344, 345, 351, 352, 353, 362, 376, 380, 381, 392, 398, 423, 424, 428, 434, 439, 441, 451, 469, 483, 501, 503, 505, 537, 577, 582, 583, 592, 594, 606, 611, 616, 630, 641, 644, 655, 663, 667, 669, 672, 706, 709, 713, 715, 736, 742, 749, 753, 761, 766, 781, 787, 791, 806, 812, 813, 814, 815, 816, 817, 840, 846, 855, 858, 890, 913, 942, 949, 957, 968, 977, 989, 1012, 1030, 1035, 1040, 1069, 1070, 1088, 1106, 1120]


In [213]:
cmp, ans = query_AND(inverted_index, 'water', 'effect')

In [214]:
print(ans)

[0, 9, 35, 44, 55, 58, 63, 67, 121, 127, 140, 175, 184, 228, 315, 324, 344, 351, 352, 362, 376, 392, 439, 441, 483, 501, 537, 594, 611, 630, 641, 663, 672, 706, 713, 715, 749, 781, 791, 812, 813, 814, 817, 846, 855, 977, 1070, 1120]
